<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
import utils as utils
from pyspark.sql.functions import  col

In [2]:
spark = SparkSession.builder.appName("StreamingApp").getOrCreate()

25/01/21 18:51:29 WARN Utils: Your hostname, Iness-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/01/21 18:51:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/21 18:51:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Step 1: Read and verify results

In [64]:
stops = utils.get_stops(spark)

In [65]:
df = spark.read.format("parquet").load("content/lake/processing/vehicles/data")
df.count()

3506

In [66]:
from pyspark.sql.functions import max
from datetime import datetime, timedelta
max_timestamp = df.select(max("last_timestamp")).collect()[0][0]
minutes_window = max_timestamp - timedelta(seconds=120)
minutes_window

datetime.datetime(2025, 1, 19, 16, 19, 29)

In [68]:
view = (df
 .filter(col("last_timestamp") > minutes_window)
 .join(stops, how='left', on='stop_id')
 .withColumn("distance_till_stop", utils.haversine_udf(col("last_lat"), col('last_lon'), col('stop_lat'), col('stop_lon')))
 .withColumn("stop_eta", col("distance_till_stop") / col("average_speed") *60)
 )


In [70]:
view.show()
# Para saber direcao era so fazer uma query as trips com o trip_id


+-------+--------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+-----------+----------+------------------+---------+---------+------------------+--------------------+
|stop_id|      id|             trip_id|              window|current_status|route_id|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|   distance|time_delta|     average_speed| stop_lat| stop_lon|distance_till_stop|            stop_eta|
+-------+--------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+-----------+----------+------------------+---------+---------+------------------+--------------------+
| 071269| 42|2516|2769_1_1|3|1|1620...|{2025-01-19 16:19...|    STOPPED_AT|  2769_1|2025-01-19 16:20:12|2025-01-19 16:20:48|38.817013|-9.164764|38.816902|-9.165228|0.041997276|        36|  4.19972762

In [73]:
view.filter(col('id') =='41|1205').show()


+-------+-------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+----------+----------+------------------+---------+---------+------------------+-------------------+
|stop_id|     id|             trip_id|              window|current_status|route_id|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|  distance|time_delta|     average_speed| stop_lat| stop_lon|distance_till_stop|           stop_eta|
+-------+-------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+----------+----------+------------------+---------+---------+------------------+-------------------+
| 170655|41|1205|1622_0_1_1600_162...|{2025-01-19 16:18...| IN_TRANSIT_TO|  1622_0|2025-01-19 16:18:47|2025-01-19 16:20:20| 38.76906| -9.30184| 38.77006|-9.306187|0.39301717|        93| 15.21356797987415|38.7